# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from catboost import Pool, cv, CatBoostRegressor
from bayes_opt import BayesianOptimization
from bayes_opt import BayesianOptimization as BO
import warnings
from sklearn.model_selection import * 
from sklearn.metrics import *
import os

# Load Data

In [2]:
train_raw = pd.read_csv('raw_data/train_data.csv', parse_dates=["startdate"])
test_raw = pd.read_csv('raw_data/test_data.csv', parse_dates=["startdate"])
submit = pd.read_csv('raw_data/sample_solution.csv')
target = 'contest-tmp2m-14d__tmp2m'
train_raw

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__nasa0,nmme0-tmp2m-34w__nmme0mean,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-56w__gfdl,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cancm3,nmme-prate-34w__cancm4,nmme-prate-34w__ccsm3,nmme-prate-34w__ccsm4,nmme-prate-34w__cfsv2,nmme-prate-34w__gfdl,nmme-prate-34w__gfdlflora,nmme-prate-34w__gfdlflorb,nmme-prate-34w__nasa,nmme-prate-34w__nmmemean,contest-wind-h100-14d__wind-hgt-100,nmme0-prate-56w__cancm30,nmme0-prate-56w__cancm40,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__gfdlflora0,nmme0-prate-56w__gfdlflorb0,nmme0-prate-56w__gfdl0,nmme0-prate-56w__nasa0,nmme0-prate-56w__nmme0mean,nmme0-prate-34w__cancm30,nmme0-prate-34w__cancm40,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflora0,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__gfdl0,nmme0-prate-34w__nasa0,nmme0-prate-34w__nmme0mean,contest-tmp2m-14d__tmp2m,contest-slp-14d__slp,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-56w__cancm3,nmme-prate-56w__cancm4,nmme-prate-56w__ccsm3,nmme-prate-56w__ccsm4,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdl,nmme-prate-56w__gfdlflora,nmme-prate-56w__gfdlflorb,nmme-prate-56w__nasa,nmme-prate-56w__nmmemean,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__ccsm3,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__nmmemean,contest-prwtr-eatm-14d__prwtr,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-precip-14d__precip,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,contest-wind-h500-14d__wind-hgt-500,cancm30,cancm40,ccsm30,ccsm40,cfsv20,gfdlflora0,gfdlflorb0,gfdl0,nasa0,nmme0mean,climateregions__climateregion,elevation__elevation,wind-vwnd-250-2010-1,wind-vwnd-250-2010-2,wind-vwnd-250-2010-3,wind-vwnd-250-2010-4,wind-vwnd-250-2010-5,wind-vwnd-250-2010-6,wind-vwnd-250-2010-7,wind-vwnd-250-2010-8,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-11,wind-vwnd-250-2010-12,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-15,wind-vwnd-250-2010-16,wind-vwnd-250-2010-17,wind-vwnd-250-2010-18,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-3,wind-uwnd-250-2010-4,wind-uwnd-250-2010-5,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-8,wind-uwnd-250-2010-9,wind-uwnd-250-2010-10,wind-uwnd-250-2010-11,wind-uwnd-250-2010-12,wind-uwnd-250-2010-13,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-16,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,mjo1d__phase,mjo1d__amplitude,mei__mei,mei__meirank,mei__nip,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-4,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,sst-2010-2,sst-2010-3,sst-2010-4,sst-2010-5,sst-2010-6,sst-2010-7,sst-2010-8,sst-2010-9,sst-2010-10,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-4,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-8,wind-hgt-500-2010-9,wind-hgt-500-2010-10,icec-2010-1,icec-2010-2,icec-2010-3,icec-2010-4,icec-2010-5,icec-2010-6,icec-2010-7,icec-2010-8,icec-2010-9,icec-2010-10,wind-uwnd-925-2010-1,wind-uwnd-925-2010-2,wind-uwnd-925-2010-3,wind-uwnd-925-2010-4,wind-uwnd-925-2010

# Preprocess Data

In [3]:
from scipy.stats import iqr

def remove_outliers_tukey(data, alpha=1.5):
    '''
    Remove outliers using Tukey's method with the interquartile range (IQR).
    
    Parameters:
    data (numpy array or pandas dataframe): The data to remove outliers from.
    alpha (float): The sensitivity parameter, which determines the range to consider outliers.
                   A value of 1.5 is the default, which is a commonly used value.
    
    Returns:
    numpy array or pandas dataframe: The data with outliers removed.
    '''
    # Select only the numerical columns
    num_cols = data.select_dtypes(include=[np.number]).columns
    data_num = data[num_cols]
    
    # Compute the first and third quartiles
    q1, q3 = np.percentile(data_num, [25, 75])
    
    # Compute the interquartile range (IQR)
    iqr_val = iqr(data_num)
    
    # Compute the range outside of which data points are considered outliers
    outlier_range = (q1 - alpha * iqr_val, q3 + alpha * iqr_val)
    
    # Identify the outliers and remove them
    outliers = (data_num < outlier_range[0]) | (data_num > outlier_range[1])
    data_num_no_outliers = data_num[~outliers]
    
    # Merge the numerical columns back into the original data frame
    data_no_outliers = pd.concat([data_num_no_outliers, data.select_dtypes(exclude=[np.number])], axis=1)
    return data_no_outliers

data_with_removed_outliers = remove_outliers_tukey(train_raw)
data_with_removed_outliers.shape

(375734, 246)

In [4]:
def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

def location_nom(train, test):
    # Ref: https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved
    scale = 14

    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)

    all_df = pd.concat([train, test], axis=0)
    all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
    train = all_df.iloc[:len(train)]
    test = all_df.iloc[len(train):].drop(target, axis=1)
    return train, test

def fill_na(df):
    col_na = ['nmme0-tmp2m-34w__ccsm30', 'nmme-tmp2m-56w__ccsm3', 'nmme-prate-34w__ccsm3', 'nmme0-prate-56w__ccsm30', 
                'nmme0-prate-34w__ccsm30', 'nmme-prate-56w__ccsm3', 'nmme-tmp2m-34w__ccsm3', 'ccsm30']

    g_means =  ['nmme0-tmp2m-34w__nmme0mean', 'nmme-tmp2m-56w__nmmemean', 'nmme-prate-34w__nmmemean', 'nmme0-prate-56w__nmme0mean', 
                'nmme0-prate-34w__nmme0mean', 'nmme-prate-56w__nmmemean', 'nmme-tmp2m-34w__nmmemean', 'nmme0mean']

    g_1 = ['nmme0-tmp2m-34w__cancm30', 'nmme0-tmp2m-34w__cancm40', 'nmme0-tmp2m-34w__ccsm40', 'nmme0-tmp2m-34w__cfsv20',
            'nmme0-tmp2m-34w__gfdlflora0', 'nmme0-tmp2m-34w__gfdlflorb0', 'nmme0-tmp2m-34w__gfdl0', 'nmme0-tmp2m-34w__nasa0']

    g_2 = ['nmme-tmp2m-56w__cancm3', 'nmme-tmp2m-56w__cancm4', 'nmme-tmp2m-56w__ccsm4', 'nmme-tmp2m-56w__cfsv2',
        'nmme-tmp2m-56w__gfdl', 'nmme-tmp2m-56w__gfdlflora', 'nmme-tmp2m-56w__gfdlflorb', 'nmme-tmp2m-56w__nasa']

    g_3 = ['nmme-prate-34w__cancm3', 'nmme-prate-34w__cancm4', 'nmme-prate-34w__ccsm4', 'nmme-prate-34w__cfsv2', 
        'nmme-prate-34w__gfdl', 'nmme-prate-34w__gfdlflora', 'nmme-prate-34w__gfdlflorb', 'nmme-prate-34w__nasa']

    g_4 = [ 'nmme0-prate-56w__cancm30', 'nmme0-prate-56w__cancm40', 'nmme0-prate-56w__ccsm40', 'nmme0-prate-56w__cfsv20',
        'nmme0-prate-56w__gfdlflora0', 'nmme0-prate-56w__gfdlflorb0', 'nmme0-prate-56w__gfdl0', 'nmme0-prate-56w__nasa0']

    g_5 = ['nmme0-prate-34w__cancm30', 'nmme0-prate-34w__cancm40', 'nmme0-prate-34w__ccsm40', 'nmme0-prate-34w__cfsv20',
        'nmme0-prate-34w__gfdlflora0', 'nmme0-prate-34w__gfdlflorb0', 'nmme0-prate-34w__gfdl0', 'nmme0-prate-34w__nasa0']

    g_6 = ['nmme-prate-56w__cancm3', 'nmme-prate-56w__cancm4', 'nmme-prate-56w__ccsm4', 'nmme-prate-56w__cfsv2',
        'nmme-prate-56w__gfdl', 'nmme-prate-56w__gfdlflora', 'nmme-prate-56w__gfdlflorb', 'nmme-prate-56w__nasa']

    g_7 = ['nmme-tmp2m-34w__cancm3', 'nmme-tmp2m-34w__cancm4', 'nmme-tmp2m-34w__ccsm4', 'nmme-tmp2m-34w__cfsv2', 
        'nmme-tmp2m-34w__gfdl','nmme-tmp2m-34w__gfdlflora', 'nmme-tmp2m-34w__gfdlflorb', 'nmme-tmp2m-34w__nasa']

    g_8 = ['cancm30', 'cancm40', 'ccsm40', 'cfsv20', 'gfdlflora0', 'gfdlflorb0', 'gfdl0', 'nasa0']

    gs = [g_1, g_2, g_3, g_4, g_5, g_6, g_7, g_8]
    zip_cols = zip(col_na, gs, g_means)
    for c, g, m in tqdm(zip_cols):
        df[c] = (df[m]*9) - df[g].sum(1)
    return df

def categorical_encode(train, test, cat_cols):
    le = LabelEncoder()
    for col in cat_cols:
        train[col] = le.fit_transform(train[[col]])
        test[col] = le.transform(test[[col]])
    return train, test

def creat_new_featute(df):
    df['year'] = df['startdate'].dt.year
    df['month'] = df['startdate'].dt.month
    df['day'] = df['startdate'].dt.day
    return df

def identify_correlated(df, threshold):
    corr_matrix = df.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    reduced_corr_matrix = corr_matrix.mask(mask)
    features_to_drop = [c for c in reduced_corr_matrix.columns if any(reduced_corr_matrix[c] > threshold)]
    return features_to_drop

def feature_scale(train, test):
    scaler = MinMaxScaler()
    train_ = scaler.fit_transform(train)
    test_ = scaler.transform(test)
    train = pd.DataFrame(train_, columns = train.columns)
    test = pd.DataFrame(test_, columns = test.columns)
    return train, test

def feature_engineering(train_raw, test_raw):
    train, test = location_nom(train_raw, test_raw)
    train = fill_na(train)
    train = creat_new_featute(train)
    test = creat_new_featute(test)
    train, test = categorical_encode(train, test, cat_cols=['climateregions__climateregion'])
    
    to_drop = identify_correlated(train.copy(), threshold=.85)
    drop_cols = ['index', 'startdate', target]
    drop_cols.extend(to_drop)
    features = [col for col in train.columns if col not in drop_cols]
    
    X = train[features]
    y = train[target]
    X_test = test[features]
    X, X_test = feature_scale(X, X_test)
    
    return X, y, X_test

# Model

In [5]:
X, y, X_test = feature_engineering(train_raw.copy(), test_raw.copy())
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2)
print(f'Train_shape: {X_train.shape}    |   Val_shape: {X_val.shape}    |   Test_shape: {X_test.shape}')

0it [00:00, ?it/s]

Train_shape: (300587, 158)    |   Val_shape: (75147, 158)    |   Test_shape: (31354, 158)


In [14]:
cb_params = {'iterations': 50000,
            'learning_rate': 0.01,
            'depth': 6,
            'l2_leaf_reg': 3,
            'bagging_temperature': 0,
            'border_count': 128,
            'loss_function': 'RMSE',
            'random_seed': 2,
            'task_type': 'GPU',
            'verbose': 500}

model = CatBoostRegressor(**cb_params)
model.fit(X_train, y_train,  eval_set=(X_val, y_val))
submit[target] = model.predict(X_test)

0:	learn: 9.7772351	test: 9.8080551	best: 9.8080551 (0)	total: 8.1ms	remaining: 6m 44s
500:	learn: 1.6515860	test: 1.6510317	best: 1.6510317 (500)	total: 3.54s	remaining: 5m 50s
1000:	learn: 1.3496983	test: 1.3531482	best: 1.3531482 (1000)	total: 7.04s	remaining: 5m 44s
1500:	learn: 1.1804714	test: 1.1851192	best: 1.1851192 (1500)	total: 10.6s	remaining: 5m 42s
2000:	learn: 1.0579464	test: 1.0637798	best: 1.0637798 (2000)	total: 14.1s	remaining: 5m 38s
2500:	learn: 0.9725057	test: 0.9790951	best: 0.9790951 (2500)	total: 17.7s	remaining: 5m 36s
3000:	learn: 0.9059109	test: 0.9131820	best: 0.9131820 (3000)	total: 21.2s	remaining: 5m 32s
3500:	learn: 0.8525503	test: 0.8606099	best: 0.8606099 (3500)	total: 24.8s	remaining: 5m 29s
4000:	learn: 0.8123727	test: 0.8212512	best: 0.8212512 (4000)	total: 28.3s	remaining: 5m 25s
4500:	learn: 0.7760531	test: 0.7852422	best: 0.7852422 (4500)	total: 31.8s	remaining: 5m 21s
5000:	learn: 0.7434435	test: 0.7529876	best: 0.7529876 (5000)	total: 35.3s	rem

In [166]:
import statsmodels.api as sm

# Fit a robust linear regression model
rlm_model = sm.RLM(y_train, X_train, M=sm.robust.norms.Hampel())
rlm_results = rlm_model.fit(scale_est=sm.robust.scale.HuberScale())

# Print the summary of the model
print(rlm_results.summary())

# Get the predicted values on the test set
y_pred = rlm_results.predict(X_val)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("RMSE:", rmse)

                                            Robust linear Model Regression Results                                            
Dep. Variable:                                               contest-tmp2m-14d__tmp2m   No. Observations:               300587
Model:                                                                            RLM   Df Residuals:                   300429
Method:                                                                          IRLS   Df Model:                          157
Norm:                                                                          Hampel                                         
Scale Est.:        <statsmodels.robust.scale.HuberScale object at 0x000001A36647E6D0>                                         
Cov Type:                                                                          H1                                         
Date:                                                                Wed, 01 Mar 2023                          

In [167]:
accuracy(y_val, y_pred)

R2 Accuracy: 0.9674275813765137
MSE Accuracy: 3.1875116420997487
MAE Accuracy: 1.3604357139899825
RMSE Accuracy: 1.7853603675728182


In [169]:
y_pred_rlm = rlm_results.predict(X_test)

## Accuracy

In [15]:
y_hat = model.predict(X_val)

In [16]:
def accuracy(y_test,y_hat):
    acc = r2_score(y_test, y_hat)
    print("R2 Accuracy:", acc)
    accMSE = mean_squared_error(y_test, y_hat)
    print("MSE Accuracy:", accMSE)
    accMAE = mean_absolute_error(y_test, y_hat)
    print("MAE Accuracy:", accMAE)
    accRMSE = mean_squared_error(y_test, y_hat, squared=False)
    print("RMSE Accuracy:", accRMSE)

In [17]:
accuracy(y_val, y_hat)

R2 Accuracy: 0.9991036818564324
MSE Accuracy: 0.08771299886176931
MAE Accuracy: 0.2300606154293133
RMSE Accuracy: 0.2961638041047037


In [48]:
xg_df = pd.read_csv('predictions/prediction4.csv')
cat_df = submit.copy(deep=True)
lgb_df = pd.read_csv('y_pred_lgb.csv')
frf_df = pd.read_csv('rfrpredictions.csv')

In [49]:
y_pred_xgb = xg_df['contest-tmp2m-14d__tmp2m']
y_pred_cat = cat_df['contest-tmp2m-14d__tmp2m']
y_pred_lgb = lgb_df['contest-tmp2m-14d__tmp2m']
y_pred_rfr = frf_df['contest-tmp2m-14d__tmp2m']

In [170]:
ensemble_preds = y_pred_lgb*0.51+y_pred_cat*0.42+y_pred_xgb*0.07+0.01*y_pred_rlm

In [171]:
ensemble_preds

0        28.700123
1        28.689365
2        28.615168
3        28.640197
4        28.610255
           ...    
31349     6.365220
31350     6.122401
31351     5.433634
31352     5.901380
31353     5.964171
Length: 31354, dtype: float64

In [165]:
train2 = pd.DataFrame(ensemble_preds)
train2['cat'] = y_pred_cat
train2['lgb'] = y_pred_lgb
train2['rfr'] = y_pred_rfr
train2['xgb'] = y_pred_xgb
train2.head()

,contest-tmp2m-14d__tmp2m,cat,lgb,rfr,xgb
0,28.396986,28.322415,28.697557,25.108389,26.654529
1,28.388082,28.336734,28.653199,25.060670,26.764611
2,28.315627,28.251097,28.572023,25.079315,26.834774
3,28.340548,28.299905,28.589340,24.905282,26.771782
4,28.311366,28.328242,28.504263,24.877544,26.804726


In [172]:
submit[target] = ensemble_preds

In [173]:
submit

,contest-tmp2m-14d__tmp2m,index
0,28.700123,375734
1,28.689365,375735
2,28.615168,375736
3,28.640197,375737
4,28.610255,375738
...,...,...
31349,6.365220,407083
31350,6.122401,407084
31351,5.433634,407085
31352,5.901380,407086


# Submission

In [174]:
files = os.listdir('predictions')
if files is not None:
    highest = 0
    for i in files:
        curr = i[10:-4]
        highest = int(curr) if int(curr)>highest else highest

    file_name = f'prediction{highest+1}.csv'
else:
    file_name = 'prediction1.csv'

In [175]:
check = submit.shape[0]==31354
check

True

In [176]:
file_name

'prediction11.csv'

In [177]:
if check:
    submit.to_csv(f"predictions/{file_name}",index=False)
else:
    print('Shape does not match requirements')